In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from tensorflow.keras.models import load_model

In [ ]:
def preprocess_data(input_csv):
    df = pd.read_csv(input_csv)
    
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], errors='coerce')
    df['date_ordinal'] = df['tpep_pickup_datetime'].apply(lambda x: x.toordinal() if pd.notnull(x) else None)
    
    df['pickup_latitude'] = pd.to_numeric(df['pickup_latitude'], errors='coerce')
    df['pickup_longitude'] = pd.to_numeric(df['pickup_longitude'], errors='coerce')
    
    df = df.dropna(subset=['date_ordinal', 'pickup_latitude', 'pickup_longitude'])
    
    return df

In [ ]:
def prepare_features(df):    
    X = df[['date_ordinal', 'pickup_latitude', 'pickup_longitude']].values
    X = X.reshape((X.shape[0], 1, X.shape[1]))
    return X

In [ ]:
def predict_weather(input_csv, output_csv):
    df = preprocess_data(input_csv)
    X = prepare_features(df)
    
    model_snow = load_model("nn_model_SNOW.h5")
    model_temp = load_model("nn_model_TAVG.h5")
    model_prcp = load_model("nn_model_PRCP.h5")
    
    df['SNOW_pred'] = model_snow.predict(X).flatten()
    df['TAVG_pred'] = model_temp.predict(X).flatten()
    df['PRCP_pred'] = model_prcp.predict(X).flatten()
        
    df.to_csv(output_csv, index=False)
    print(f"Predictions saved to {output_csv}")

In [ ]:
predict_weather(r"../Datasets/nyc_taxi_sample.csv", r"../Datasets/fin_ds.csv")

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Predictions saved to Datasets\fin_ds.csv
